# NVCU Database Generator

By Kenneth Burchfiel

Released under the MIT License

This script will create a SQLite database for the (fictional) Northern Virginia Catholic University that can be referenced within other sections of Python for Nonprofits. The data within this database will be fictional also.

In [1]:
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
# Setting up random number generation capabilities:
rng = np.random.default_rng(2325) 
# Based on https://numpy.org/doc/stable/reference/random/generator.html 
# The faker library is a great tool for creating fictional
# database records. I set the locale to 'en_US' because NVCU
# is based in the United States.
from faker import Faker; fake = Faker('en_US')  



## Connecting to our NVCU database via the SQLAlchemy library

(This code will work even if no database currently exists at the path shown below.)


In [2]:
e = create_engine('sqlite:///nvcu_db.db')
# Based on: https://docs.sqlalchemy.org/en/20/dialects/sqlite.html#pysqlite


In [3]:
student_count = 2**14
student_count

16384

# Creating a current enrollment table



## Creating lists of names

We'll use the Faker library to create equal numbers of female and male first names, then concatenate these lists to create a single list of first names.

(Note: The [Faker documentation for the 'en_US' (US English) locale](https://faker.readthedocs.io/en/master/locales/en_US.html) was a useful resource in writing this code.)

In [4]:
student_count
# It's convenient to also create a variable for the value equal
# to half of the student count. (Dividing the student count by
# 2 produces a float by default, so we'll convert this value to 
# an int so that it can be used as a range within list comprehensions.)
half_student_count = int(student_count / 2)
half_student_count

8192

In [5]:
female_first_names = [
    fake.first_name_female() 
    for i in range(half_student_count)]
male_first_names = [
    fake.first_name_male() for i in range(half_student_count)]
first_names = female_first_names + male_first_names
last_names = [
    fake.last_name() for i in range(student_count)]

In [6]:
# In order to make our genders match our names, we'll 
# make the first half of our gender list female and the second 
# half male (as the first and second halves of our first names
# list show male and female names, respectively.)
genders = (['F' for i in range(half_student_count)] 
+ ['M' for i in range(half_student_count)])

## Initializing our current enrollment table

In [7]:
df_curr_enrollment = pd.DataFrame(
    index = np.arange(0,student_count), 
data = {'first_name':first_names,
        'last_name':last_names,
        'gender':genders})
df_curr_enrollment

,first_name,last_name,gender
0,Danielle,Wilson,F
1,Anna,Smith,F
2,Ann,Baker,F
3,Lori,Webb,F
4,Christine,Hunter,F
...,...,...,...
16379,Patrick,Patterson,M
16380,Randall,Davidson,M
16381,Danny,Brandt,M
16382,David,Clark,M


Creating matriculation years:

In order to simulate increasing enrollment over time, weights were added to the rng.choice() call so that recent years would appear more frequently.

In [8]:
rng.choice(
    [2020, 2021, 2022, 2023], 
    p=[0.20, 0.22, 0.25, 0.33], size = student_count)

array([2023, 2023, 2020, ..., 2023, 2022, 2023])

In [9]:
df_curr_enrollment['matriculation_year'] = rng.choice(
    [2020, 2021, 2022, 2023], p=[0.20, 0.22, 0.25, 0.33], 
    size=student_count)
# https://numpy.org/doc/stable/reference/random/generated/numpy.random.Generator.choice.html
df_curr_enrollment

,first_name,last_name,gender,matriculation_year
0,Danielle,Wilson,F,2020
1,Anna,Smith,F,2020
2,Ann,Baker,F,2020
3,Lori,Webb,F,2022
4,Christine,Hunter,F,2023
...,...,...,...,...
16379,Patrick,Patterson,M,2020
16380,Randall,Davidson,M,2020
16381,Danny,Brandt,M,2022
16382,David,Clark,M,2020


## Creating student IDs
Student IDs will use the format matriculation_year-matriculation_number. matriculation_number represents the order in which students enrolled for a given year; these numbers are unique within each year, but not between years. This number can then be combined with students' matriculation years to form a unique ID.

In [10]:
# Calculating matriculation numbers by grouping the DataFrame by
# matriculation year, then assigning each student within each year a unique
# number:
# (This can be achieved via df.groupby() and df.rank(). See
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.groupby.html
# and https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.rank.html )
df_curr_enrollment['matriculation_number'] = df_curr_enrollment.groupby(
    'matriculation_year')['matriculation_year'].rank(
    method = 'first').astype('int')
df_curr_enrollment

,first_name,last_name,gender,matriculation_year,matriculation_number
0,Danielle,Wilson,F,2020,1
1,Anna,Smith,F,2020,2
2,Ann,Baker,F,2020,3
3,Lori,Webb,F,2022,1
4,Christine,Hunter,F,2023,1
...,...,...,...,...,...
16379,Patrick,Patterson,M,2020,3287
16380,Randall,Davidson,M,2020,3288
16381,Danny,Brandt,M,2022,3999
16382,David,Clark,M,2020,3289


In [11]:
# Adding matriculation years, matriculation numbers, and hyphens together
# to create student IDs:
df_curr_enrollment['student_id'] = (
    df_curr_enrollment['matriculation_year'].astype('str') 
    + '-' 
    + df_curr_enrollment['matriculation_number'].astype('str'))
# Sorting the DataFrame by matriculation year and matriculation number:
# (The DataFrame could also be sorted by student_id, but because these numbers
# would be interpreted as strings, ids like 2020-10 would appear 
# in front of ones like 2020-2.)
df_curr_enrollment.sort_values(
    ['matriculation_year', 'matriculation_number'], inplace = True)
df_curr_enrollment.reset_index(drop=True,inplace=True)

df_curr_enrollment

,first_name,last_name,gender,matriculation_year,matriculation_number,student_id
0,Danielle,Wilson,F,2020,1,2020-1
1,Anna,Smith,F,2020,2,2020-2
2,Ann,Baker,F,2020,3,2020-3
3,Jennifer,Costa,F,2020,4,2020-4
4,Brenda,Wood,F,2020,5,2020-5
...,...,...,...,...,...,...
16379,Richard,Ellison,M,2023,5439,2023-5439
16380,Johnny,Henry,M,2023,5440,2023-5440
16381,Mark,Trevino,M,2023,5441,2023-5441
16382,Thomas,Rose,M,2023,5442,2023-5442


We'll also use matriculation years as a basis for dates of birth. (These date of birth calculations aren't the most realistic, but they are good enough for use within our fictional dataset.)

In [12]:
df_curr_enrollment['birth_month'] = rng.integers(
    low = 1, high = 13, size = student_count).astype('str')
df_curr_enrollment['birth_day'] = rng.integers(
    low = 1, high = 29, size = student_count).astype('str')

# Adding leading zeros to month and date values:
for column in ['birth_month', 'birth_day']:
    df_curr_enrollment[column] = df_curr_enrollment[column].str.zfill(2)

# In order to prevent a non-existent date (e.g. February 31) from getting stored
# in our results, we'll limit our birthday integers to the range [1, 28].
df_curr_enrollment['birth_year'] = (df_curr_enrollment[
'matriculation_year'] - 18).astype('str')

# Combining these columns together to create date of birth values in
# YYYY-MM-DD format:
df_curr_enrollment['date_of_birth'] = (
    df_curr_enrollment['birth_year'] + '-'
    + df_curr_enrollment['birth_month'] + '-'
    + df_curr_enrollment['birth_day'])

# Now that we have our date_of_birth column in place, we no longer need
# the columns that stored individual parts of those birthdays:
df_curr_enrollment.drop(
    ['birth_month', 'birth_day', 
    'birth_year'], axis = 1, inplace = True)

df_curr_enrollment

,first_name,last_name,gender,matriculation_year,matriculation_number,student_id,date_of_birth
0,Danielle,Wilson,F,2020,1,2020-1,2002-12-16
1,Anna,Smith,F,2020,2,2020-2,2002-09-26
2,Ann,Baker,F,2020,3,2020-3,2002-05-18
3,Jennifer,Costa,F,2020,4,2020-4,2002-11-15
4,Brenda,Wood,F,2020,5,2020-5,2002-06-21
...,...,...,...,...,...,...,...
16379,Richard,Ellison,M,2023,5439,2023-5439,2005-03-26
16380,Johnny,Henry,M,2023,5440,2023-5440,2005-12-17
16381,Mark,Trevino,M,2023,5441,2023-5441,2005-09-13
16382,Thomas,Rose,M,2023,5442,2023-5442,2005-11-03


## Assigning students to different colleges

(I used Wikipedia's ['List of patron saints by occupation and activity'](https://en.wikipedia.org/wiki/List_of_patron_saints_by_occupation_and_activity) page to determine the saint after which each college would be named.)

NVCU has four different colleges:

1. St. Luke's, a humanities college. (St. Luke is one of the patron saints of artists.) Abbreviation: STL
2. St. Benedict's, a STEM college. (St. Benedict is one of the patron saints of engineers.) Abbreviation: STB
3. St. Matthew's, a business college. (St. Matthew is the patron saint of accountants.) Abbreviation: STM
4. St. Catherine's, a health college. (St. Catherine of Alexandria is one of the patron saints of nurses.) Abbreviation: STC

We can use np.choice to assign students to different colleges. However, to make the data more interesting, we'll have one college (STL) increase in popularity over time; another (STC) decrease in popularity; and the two remaining colleges remain roughly constant in popularity. We can simulate these changes by (1) creating filtered versions of the DataFrame for each year; (2) calling np.choice() with different probability sets for each year in order to create the 'college' column; and (3) recreating df_curr_enrollment by adding these year-specific DataFrames back together.

In [13]:
df_list = []
for year in df_curr_enrollment['matriculation_year'].unique():
    print(f"Now adding in college enrollments for {year}.")
    df = df_curr_enrollment.query("matriculation_year == @year").copy()
    if year == 2020:
        probabilities = [0.15, 0.25, 0.3, 0.3]
    elif year == 2021:
        probabilities = [0.19, 0.26, 0.29, 0.26]
    elif year == 2022:
        probabilities = [0.25, 0.27, 0.27, 0.21]
    elif year == 2023:
        probabilities = [0.27, 0.25, 0.31, 0.17]
    else:
        raise ValueError(
            f"A probability list needs to be added in for {year}.")
    df['college'] = rng.choice(
        ['STL', 'STB', 'STM', 'STC'], 
        p = probabilities, size = len(df))
    df_list.append(df)
df_curr_enrollment = pd.concat([df for df in df_list])

Now adding in college enrollments for 2020.
Now adding in college enrollments for 2021.
Now adding in college enrollments for 2022.
Now adding in college enrollments for 2023.


The commented-out cell below shows an alternative approach to assigning colleges to each student. Because it iterates through each row in the DataFrame, it took 2.25 seconds to run on my laptop versus 0.016 seconds for the method shown above; in other words, the method above was around 133 times faster.

In [14]:
# df_curr_enrollment['college'] = ''
# college_col = df_curr_enrollment.columns.get_loc('college')
# for i in range(len(df_curr_enrollment)):
#     year = df_curr_enrollment.iloc[i]['matriculation_year']
#     if year == 2020:
#         probabilities = [0.15, 0.25, 0.3, 0.3]
#     elif year == 2021:
#         probabilities = [0.19, 0.26, 0.29, 0.26]
#     elif year == 2022:
#         probabilities = [0.25, 0.27, 0.27, 0.21]
#     elif year == 2022:
#         probabilities = [0.27, 0.25, 0.31, 0.17]
#     df_curr_enrollment.iloc[i, college_col] = rng.choice(
#     ['STL', 'STB', 'STM', 'STC'], p = probabilities)

In [15]:
df_curr_enrollment

,first_name,last_name,gender,matriculation_year,matriculation_number,student_id,date_of_birth,college
0,Danielle,Wilson,F,2020,1,2020-1,2002-12-16,STC
1,Anna,Smith,F,2020,2,2020-2,2002-09-26,STM
2,Ann,Baker,F,2020,3,2020-3,2002-05-18,STC
3,Jennifer,Costa,F,2020,4,2020-4,2002-11-15,STC
4,Brenda,Wood,F,2020,5,2020-5,2002-06-21,STM
...,...,...,...,...,...,...,...,...
16379,Richard,Ellison,M,2023,5439,2023-5439,2005-03-26,STL
16380,Johnny,Henry,M,2023,5440,2023-5440,2005-12-17,STM
16381,Mark,Trevino,M,2023,5441,2023-5441,2005-09-13,STL
16382,Thomas,Rose,M,2023,5442,2023-5442,2005-11-03,STB


In [16]:
## Assigning additional year-related values:

In [17]:
df_curr_enrollment['class_of'] = df_curr_enrollment['matriculation_year'] + 4
df_curr_enrollment['level'] = df_curr_enrollment['matriculation_year'].map(
    {2020:'Fr',2021:'So',
     2022:'Ju',2023:'Se'}) # Fr, So, Ju, and Se stand for 
# Freshman, Sophomore, Junior, and Senior, respectively.
# Creating an integer-based equivalent to 'level':
df_curr_enrollment['level_for_sorting'] = (
    df_curr_enrollment['matriculation_year'] - 2020)
df_curr_enrollment

,first_name,last_name,gender,matriculation_year,matriculation_number,student_id,date_of_birth,college,class_of,level,level_for_sorting
0,Danielle,Wilson,F,2020,1,2020-1,2002-12-16,STC,2024,Fr,0
1,Anna,Smith,F,2020,2,2020-2,2002-09-26,STM,2024,Fr,0
2,Ann,Baker,F,2020,3,2020-3,2002-05-18,STC,2024,Fr,0
3,Jennifer,Costa,F,2020,4,2020-4,2002-11-15,STC,2024,Fr,0
4,Brenda,Wood,F,2020,5,2020-5,2002-06-21,STM,2024,Fr,0
...,...,...,...,...,...,...,...,...,...,...,...
16379,Richard,Ellison,M,2023,5439,2023-5439,2005-03-26,STL,2027,Se,3
16380,Johnny,Henry,M,2023,5440,2023-5440,2005-12-17,STM,2027,Se,3
16381,Mark,Trevino,M,2023,5441,2023-5441,2005-09-13,STL,2027,Se,3
16382,Thomas,Rose,M,2023,5442,2023-5442,2005-11-03,STB,2027,Se,3


## Saving this table to our NVCU database

(This operation will also create our database file (e.g. nvcu_db.db, the path specified when we first created our database engine) if it did not exist already.)

In [18]:
df_curr_enrollment.to_sql(
    'curr_enrollment', 
    con=e, if_exists='replace', index=False)

16384

To demonstrate that the above operation was successful, we can read in a copy of this table via pd.read_sql():

In [19]:
pd.read_sql('curr_enrollment', con = e)

,first_name,last_name,gender,matriculation_year,matriculation_number,student_id,date_of_birth,college,class_of,level,level_for_sorting
0,Danielle,Wilson,F,2020,1,2020-1,2002-12-16,STC,2024,Fr,0
1,Anna,Smith,F,2020,2,2020-2,2002-09-26,STM,2024,Fr,0
2,Ann,Baker,F,2020,3,2020-3,2002-05-18,STC,2024,Fr,0
3,Jennifer,Costa,F,2020,4,2020-4,2002-11-15,STC,2024,Fr,0
4,Brenda,Wood,F,2020,5,2020-5,2002-06-21,STM,2024,Fr,0
...,...,...,...,...,...,...,...,...,...,...,...
16379,Richard,Ellison,M,2023,5439,2023-5439,2005-03-26,STL,2027,Se,3
16380,Johnny,Henry,M,2023,5440,2023-5440,2005-12-17,STM,2027,Se,3
16381,Mark,Trevino,M,2023,5441,2023-5441,2005-09-13,STL,2027,Se,3
16382,Thomas,Rose,M,2023,5442,2023-5442,2005-11-03,STB,2027,Se,3


This same table can also be saved as a standalone .csv file (thus making it easier to examine via a spreadsheet editor):

In [20]:
df_curr_enrollment.to_csv('curr_enrollment.csv', index = False)
df_curr_enrollment.to_csv(
    '../Data_Retrieval/curr_enrollment_tab_separated.csv', 
    index = False, sep = '\t')

In [21]:
df_curr_enrollment.to_excel(
    '../Data_Retrieval/curr_enrollment.xlsx', index = False)

# Creating a survey results table

This table will store fall and spring student survey results for the most recent school year. We'll configure our results so that certain groups (e.g. freshmen, seniors, and students in the STB and STL colleges) report higher results over time than do others.

### Creating fall survey results

In [22]:
def limit_results(df, column, min, max):
    '''This function restricts the values within a particular DataFrame
    column to a range provided by the user.
    
    df: the DataFrame to modify
    
    column: the column within df whose values will be restricted.
    
    min and max: the minimum and max values, respectively, to allow within
    this column.
    '''
    df[column] = np.where(
        df[column] < min, min, df[column])
    df[column] = np.where(
        df[column] > max, max, df[column])

In [23]:
# Initializing the table as a copy of selected columns from df_curr_enrollment:
df_fall_survey = df_curr_enrollment.copy()[
['matriculation_year', 'matriculation_number',
 'student_id', 'college', 'level']]
df_fall_survey['starting_year'] = 2023 # Other years could be stored
# within this table as well.
df_fall_survey['season'] = 'Fall'
# We'll use rng.normal(), which produces random numbers that follow a normal
# distribution, to initialize our fall results.
# See https://numpy.org/doc/stable/reference/random/generated/numpy.random.Generator.normal.html
df_fall_survey['score'] = rng.normal(
    loc = 70, scale = 10, size = student_count).astype('int')
# calling limit_results to restrict the scores to the range [0, 100]:
limit_results(df_fall_survey, 'score', 0, 100)
df_fall_survey

,matriculation_year,matriculation_number,student_id,college,level,starting_year,season,score
0,2020,1,2020-1,STC,Fr,2023,Fall,88
1,2020,2,2020-2,STM,Fr,2023,Fall,37
2,2020,3,2020-3,STC,Fr,2023,Fall,54
3,2020,4,2020-4,STC,Fr,2023,Fall,56
4,2020,5,2020-5,STM,Fr,2023,Fall,77
...,...,...,...,...,...,...,...,...
16379,2023,5439,2023-5439,STL,Se,2023,Fall,70
16380,2023,5440,2023-5440,STM,Se,2023,Fall,96
16381,2023,5441,2023-5441,STL,Se,2023,Fall,76
16382,2023,5442,2023-5442,STB,Se,2023,Fall,67


Confirming that the minimum and maximum 'score' values within df_fall_survey lie within the range [0, 100]:

In [24]:
df_fall_survey['score'].min(), df_fall_survey['score'].max()

(30, 100)

## Creating spring results

In order to create these results, we'll first create a copy of our fall results table. Next, we'll calculate our spring results by creating a score_change variable that defaults to 10; adjusting this variable up or down based on students' college membership and level; and then adding the product of this variable and rng.random()* to a student's fall score.

We'll use .iloc[] to make these updates. An alternative would be df.at[], but the approach shown in this cell allows for different index labels to be used.


*rng.random() produces a float in the range [0, 1). See [the numpy documentation](https://numpy.org/doc/stable/reference/random/generated/numpy.random.Generator.random.html) for more details. The use of rng.random() allows for more variation in individual fall-to-spring changes.

In [25]:
df_spring_survey = df_fall_survey.copy().rename(
    columns={'score':'fall_score'}).replace('Fall','Spring')
df_spring_survey['score'] = np.NaN
score_col = df_spring_survey.columns.get_loc('score')
for i in range(len(df_spring_survey)):
    score_change = 10
    college = df_spring_survey.iloc[i]['college']
    level = df_spring_survey.iloc[i]['level']
    if college in ['STL', 'STM']:
        score_change += 20
    if level in ['So', 'Ju']:
        score_change -= 10
    # Using .iloc to make our updates:
    # This approach prevents 'setting with copy' warnings from being displayed. 
    # Note the use of get_loc to determine which column index value to 
    # incorporate into .iloc.
    df_spring_survey.iloc[i, score_col] = (df_spring_survey.iloc[
    i]['fall_score'] -5 + rng.random() * score_change)
    

limit_results(df_spring_survey, 'score', 0, 100)
df_spring_survey['score'] = df_spring_survey['score'].astype('int')
df_spring_survey.head()

,matriculation_year,matriculation_number,student_id,college,level,starting_year,season,fall_score,score
0,2020,1,2020-1,STC,Fr,2023,Spring,88,86
1,2020,2,2020-2,STM,Fr,2023,Spring,37,35
2,2020,3,2020-3,STC,Fr,2023,Spring,54,54
3,2020,4,2020-4,STC,Fr,2023,Spring,56,57
4,2020,5,2020-5,STM,Fr,2023,Spring,77,74


A pivot table of results by college and level reveals the variation in mean scores created by the above for loop.

In [26]:
df_spring_survey.pivot_table(
    index = ['college', 'level'], 
    values = 'score', aggfunc = 'mean').reset_index()

,college,level,score
0,STB,Fr,68.543287
1,STB,Ju,65.377306
2,STB,Se,69.177235
3,STB,So,64.950769
4,STC,Fr,69.586760
5,STC,Ju,64.331325
6,STC,Se,69.079348
7,STC,So,64.180932
8,STL,Fr,77.915984
9,STL,Ju,74.005123


## Combining fall and spring results into the same table

We'll use a 'long' format for this table. In other words, rather than show fall and spring results side by side for each student (an example of a 'wide'-formatted table), we'll list these results as separate rows. I consider this setup to be more realistic: in real life, this table would likely be created by stacking individual sets of survey results on top of one another.

The only identifier that we'll preserve is the student ID, as this will be sufficient for linking this dataset with any other table (such as our current enrollment dataset) that also contains this key.

In [27]:
df_survey = pd.concat(
    [df_fall_survey, df_spring_survey]).reset_index()[
['student_id', 'starting_year', 'season', 'score']]
df_survey

,student_id,starting_year,season,score
0,2020-1,2023,Fall,88
1,2020-2,2023,Fall,37
2,2020-3,2023,Fall,54
3,2020-4,2023,Fall,56
4,2020-5,2023,Fall,77
...,...,...,...,...
32763,2023-5439,2023,Spring,90
32764,2023-5440,2023,Spring,100
32765,2023-5441,2023,Spring,89
32766,2023-5442,2023,Spring,64


In [28]:
df_survey.to_sql('survey_results', if_exists = 'replace', 
                 index = False, con = e)
df_survey.to_csv('survey_results.csv', index = False)

## Creating a set of winter results that can be incorporated into a data cleaning/reformatting script

In order to demonstrate how the Pandas library can be used to clean and reformat data, I'll now create a winter survey results table that contains several issues, including:

1. Column names that differ from those in the fall/spring results
2. Different data formats
3. A missing column
4. Duplicate values
5. Missing values for certain students
6. Results spread over 16 separate files (one for each school/level pair)

These issues will be addressed within the data_cleaning_and_reformatting.ipynb script within PFN's Data Prep section.

In [29]:
df_winter_survey = df_fall_survey.copy()
df_winter_survey['SEASON'] = 'W'
# Initializing winter survey results as lower copies of fall ones, 
# then converting them to a different data type:
# (Converting the results to integers, then strings prevents the original
# decimal values from getting added to the final output.
df_winter_survey['SURVEY_SCORE'] = (
    df_winter_survey['score'] 
    + (rng.random(size = student_count) * -10)).astype(
        'int').astype('str')+'.0%'
df_winter_survey['STARTINGYR'] = df_winter_survey['starting_year'] - 2000
df_winter_survey['MATRICYR'] = (
    df_winter_survey['matriculation_year'] - 2000).astype('str')
df_winter_survey.rename(columns = {'matriculation_number':'MATRIC#'}, inplace = True)
df_winter_survey.drop(
    ['matriculation_year', 'student_id', 'starting_year',
     'season','score'], axis = 1, inplace = True)
# Removing 15% of students from the results via df.sample:      
df_winter_survey = df_winter_survey.sample(frac=0.85).copy()
# See https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sample.html
# using df.sample() to add duplicate records for 5% of the students present
# within the sample:
df_winter_survey = pd.concat(
    [df_winter_survey, df_winter_survey.sample(frac=0.05)])
df_winter_survey.reset_index(drop=True,inplace=True)
df_winter_survey

,MATRIC#,college,level,SEASON,SURVEY_SCORE,STARTINGYR,MATRICYR
0,1123,STB,Se,W,96.0%,23,23
1,2502,STM,Fr,W,57.0%,23,20
2,3725,STC,Se,W,75.0%,23,23
3,301,STC,So,W,70.0%,23,21
4,1913,STC,Fr,W,78.0%,23,20
...,...,...,...,...,...,...,...
14617,3572,STC,So,W,64.0%,23,21
14618,681,STB,Fr,W,80.0%,23,20
14619,3510,STM,Ju,W,67.0%,23,22
14620,1905,STL,Se,W,37.0%,23,23


Dividing these results into 16 individual datasets (one for each college/level pair), then saving those results into the Data Prep section of PFN:

(The 'college' and 'level' fields within df_winter_survey were only necessary for filtering data and creating school and level-based filenames, so they don't need to be included in the .csv copies of the data.)

In [30]:
for college in df_winter_survey['college'].unique():
    for level in df_winter_survey['level'].unique():
        df_results = df_winter_survey.query(
            "college == @college & level == @level").copy().drop(
            ['college', 'level'], axis = 1)
        df_results.to_csv(
            f'../Data_Prep/winter_results/{college}_{level}_results.csv', 
            index = False)

That's the end of this script for now. I may add additional tables to this database in the future, including:

1. A table that stores starting incomes for graduating seniors over the last 2+ years
2. A table that compares students' college admissions test percentiles to graduate admissions test percentiles. (This could serve as an indicator of the extent of their intellectual growth during their time at NVCU)